# Visualización de Representaciones Latentes con Autoencoder y UMAP

En este proyecto, entrenamos un autoencoder para aprender representaciones latentes de imágenes. Usamos DBSCAN para agrupar las latentes y UMAP para visualizarlas en un espacio de 2 dimensiones. También incluimos la proyección de nuevas imágenes al espacio generado.

In [2]:
#Importar las librerías

from PIL import Image, ImageDraw
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models
import umap
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors
import pandas as pd


In [3]:
# Cargamos los datos

directorio_destino = "datos10"


def cargar_imagenes(directorio):
    """
    Carga imágenes del directorio especificado y sus etiquetas.

    :param directorio: Ruta del directorio del dataset.
    :return: Arrays de imágenes y etiquetas.
    """
    imagenes = []
    etiquetas = []
    rutas = []
    clases = os.listdir(directorio)
    for clase in clases:
        clase_dir = os.path.join(directorio, clase)
        if os.path.isdir(clase_dir):
            for archivo in os.listdir(clase_dir):
                if archivo.endswith(".png"):
                    ruta = os.path.join(clase_dir, archivo)
                    imagen = Image.open(ruta).convert("L")  # Convertir a escala de grises
                    imagen = imagen.resize((64, 64))  # Redimensionar
                    imagen = np.array(imagen) / 255.0  # Normalizar
                    imagenes.append(imagen.flatten())
                    etiquetas.append(int(clase.split("_")[1]))
                    rutas.append(ruta)
    return np.array(imagenes), np.array(etiquetas), rutas


X, y, rutas = cargar_imagenes("datos10")


In [ ]:
# 3. Construcción y Entrenamiento del Autoencoder

input_dim = X.shape[1]
encoding_dim = 64  

input_img = layers.Input(shape=(input_dim,))
encoded = layers.Dense(256, activation='relu')(input_img)
encoded = layers.Dense(128, activation='relu')(encoded)
encoded = layers.Dense(encoding_dim, activation='relu')(encoded)

decoded = layers.Dense(128, activation='relu')(encoded)
decoded = layers.Dense(256, activation='relu')(decoded)
decoded = layers.Dense(input_dim, activation='sigmoid')(decoded)

autoencoder = models.Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

autoencoder.fit(X, X,
                epochs=100,  
                batch_size=256,
                shuffle=True,
                validation_split=0.2)


In [ ]:
# Extracción de Características y Clustering con DBSCAN


encoder = models.Model(input_img, encoded)
X_encoded = encoder.predict(X)


reducer = umap.UMAP(n_components=2, random_state=42)
X_umap = reducer.fit_transform(X_encoded)

dbscan = DBSCAN(eps=0.5, min_samples=10)
clusters_dbscan = dbscan.fit_predict(X_encoded)

from collections import Counter
cluster_to_squares = {
    -1: "Ruido",  
    0: "2 Cuadrados",
    1: "10 Cuadrados",
    2: "5 Cuadrados",
    3: "4 Cuadrados",
    4: "3 Cuadrados",
    5: "8 Cuadrados",
    6: "6 Cuadrados",
    7: "1 Cuadrados",
    8: "7 Cuadrados",
    9: "9 Cuadrados"
}
cluster_counts = Counter(clusters_dbscan)
cluster_labels = {cluster: f"{cluster_to_squares[cluster]} ({count})" for cluster, count in cluster_counts.items()}

plt.figure(figsize=(12, 10))
sns.scatterplot(x=X_umap[:,0], y=X_umap[:,1], hue=[cluster_labels[c] for c in clusters_dbscan], palette='tab10', legend='full')
plt.title("Clusters de Imágenes usando DBSCAN y UMAP")
plt.xlabel("UMAP 1")
plt.ylabel("UMAP 2")
plt.legend(title='Número de Cuadrados (Cantidad)', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

In [64]:
#Función para localizar nueva imagen en el UMAP

def actualizar_umap_con_imagen(nueva_imagen, encoder, dbscan, reducer, X_umap, clusters_dbscan, cluster_to_squares):
    """
    Procesa una nueva imagen, la clasifica, y actualiza el gráfico UMAP.

    :param nueva_imagen: Imagen a clasificar.
    :param encoder: Modelo encoder entrenado.
    :param dbscan: Modelo DBSCAN entrenado.
    :param reducer: Modelo UMAP entrenado.
    :param X_umap: Datos UMAP existentes.
    :param clusters_dbscan: Clusters existentes.
    :param cluster_to_squares: Diccionario que mapea clusters a números de cuadrados.
    """
    # Preprocesar la nueva imagen
    nueva_imagen = nueva_imagen.resize((64, 64)).convert("L")
    nueva_imagen = np.array(nueva_imagen) / 255.0
    nueva_imagen_flat = nueva_imagen.flatten().reshape(1, -1)
    
    # Codificar la imagen
    encoded_img = encoder.predict(nueva_imagen_flat)
    
    # Predecir el cluster
    nuevo_cluster = dbscan.fit_predict(encoded_img)[0]
    num_cuadrados = cluster_to_squares.get(nuevo_cluster, "Desconocido")
    
    # Reducir dimensionalidad con UMAP
    nueva_imagen_umap = reducer.transform(encoded_img)
    
    # Actualizar el gráfico UMAP
    plt.figure(figsize=(12, 10))
    sns.scatterplot(x=X_umap[:,0], y=X_umap[:,1], hue=[cluster_labels[c] for c in clusters_dbscan], palette='tab10', legend='full')
    plt.scatter(nueva_imagen_umap[:,0], nueva_imagen_umap[:,1], color='red', label=f'Nueva Imagen: {num_cuadrados} Cuadrados')
    plt.title("Clusters de Imágenes usando DBSCAN y UMAP")
    plt.xlabel("UMAP 1")
    plt.ylabel("UMAP 2")
    plt.legend(title='Número de Cuadrados (Cantidad)', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.show()

In [ ]:
# Selección y colocado de la nueva imagen
nueva_imagen = Image.open('imagen_13.png')
actualizar_umap_con_imagen(nueva_imagen, encoder, dbscan, reducer, X_umap, clusters_dbscan, cluster_to_squares)

In [ ]:
# Guardar el autoencoder y el encoder
autoencoder.save('autoencoder_model10.h5')
encoder.save('encoder_model10.h5')